In [ ]:
import logging
import random
from itertools import chain, product, starmap
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import Dict, List, Callable
from xml.etree import ElementTree

import numpy as np
import requests
import xmltodict
from toolz import *
import math

from collections import Counter

%load_ext autoreload
%autoreload 2

**Multi-paradigm programming in Python**

Elias Mistler | Machine Learning Engineer

[Previse](https://previ.se/)

https://github.com/eliasmistler/europython2020-multi-paradigm-sudoku

**Quick Intro**
* Elias Mistler
* Previse
    * Invoice financing
    * based on ML
    * corporate data
    * improve SME cashflow
* Machine Learning Engineer
    * ML integration into invoice processing platform
    * Buyer data intake and mapping
    * Operational tooling

# Contents

* Introduction
* Code Structure
* Data Structures
* State Handling
* Multiple implementations
* Summary

# Introduction
* Python = multi-paradigm (unlike OO Java / FP Clojure)
* OOP and FP are **concepts**, not tied to syntax (`class` or `def`)

## Object-oriented principles
* mutable data structures
* (relies on rich type system)
* class hierarchies
    * inheritance
    * abstraction
    * encapsulation
    * polymorphism

## Functional programming Principles

* immutable data structures
* (relies on simple data types)
* pure functions
    * no side-effects
    * idempotent
* (often lazy evaluation)

## Sudoku
<img src="./img/sudoku.png" style="width: 400px; float: left"/>

* 9 x 9 grid
* numbers from 1 - 9
* each row/column/block should contain each digit

# Code structure: high- vs. low-context
Example: parse raw Sudoku string (from [OpenSudoku](https://opensudoku.moire.org/)) to array

In [ ]:
raw_example = '700150000003002097800470126500390200030010050008027001975031004120700900000065002'

## Factory function (OO)

In [ ]:
@dataclass
class Sudoku:
    grid: np.array

    @classmethod
    def from_string(cls, raw):
        values = []
        for digit in raw:
            values.append(int(digit))
        grid = np.array(values, dtype='int64').reshape((9, 9))
        return cls(grid)

In [ ]:
Sudoku.from_string(raw_example)

* explicit, high-context
* easy to find and use

## Isolated function (FP)

In [ ]:
def parse_raw(raw):
    return np.array(list(map(int, raw)), dtype='int64').reshape((9, 9))

In [ ]:
parse_raw(raw_example)

* free of assumptions about the use case
* easy to reuse or generalise

## Multi-paradigm solution
Generalised, low-context pure function, use in high-context class

In [ ]:
def parse_raw(raw):
    size = int(math.sqrt(len(raw)))
    return np.array(list(map(int, raw)), dtype='int64').reshape((size, size))
    
    
class Sudoku:
    @classmethod
    def from_string(cls, raw):
        values = parse_raw(raw)
        return cls(values)

* low-context pure functions *and* high-context class
* tidy, reusable code
* generalises well
* works in any context
* easy to use and explore

## That tedious `for`-loop

In [ ]:
values = []
for digit in raw_example:
    values.append(int(digit))

values[:5]

<img src="https://www.profkrg.com/wp-content/uploads/2014/10/I-would-have-written-a-shorter-letter.png" style="max-height: 600px; float: left"/>

* easy to write
* tedious to read and reconstruct
* comparatively far from high-level intention
* error prone

The alternative:

In [ ]:
values = tuple(map(int, raw_example))

values[:5]

In [ ]:
values = thread_last(raw_example, (map, int), tuple)

* concise
* reflects the intention
* easy to read
* can take longer to write

Also useful: *list comprehension*

In [ ]:
values = [int(digit) for digit in raw_example]

* Pythonic middle ground
* easy to both read *and* write
* do not use *lambda functions* in list comprehensions!
* combine with pure functions for best results

## Further example - display/format

### Object-oriented
Implement `__repr__`

In [ ]:
from sudoku.oo.base import Sudoku

Sudoku.from_string(raw_example)

### Functional
explicit functions

In [ ]:
from sudoku.fp.load import format_sudoku

thread_last(raw_example, parse_raw, format_sudoku, print)

### Multi-paradigm
<img src="./img/why-not-both.jpg" style="width: 250px;"/>

In [ ]:
def format_sudoku(grid):
    ...


class Sudoku:
    ...
    
    def __repr__(self):
        return format_sudoku(self.grid)

# Data Structures: explicit vs. minimalist
Example: The Sudoku grid

## Class Hierarchy (OO)

<img src="./img/erm.png" style="max-height: 400px;"/>

In [ ]:
from sudoku.oo.base import *

oo_game = Sudoku.from_string(raw_example)
oo_game.get_row(8)

In [ ]:
oo_game.get_square(8, 4)

* assumes certain usage patterns
* intuitive to explore
* fairly rigid
* requires lots of boilerplate

In [ ]:
# even with `dataclass` and without many getters, setters etc:
!wc ./sudoku/oo/base.py

## Simplicity (FP)

In [ ]:
import schema

sudoku_schema = schema.And(np.ndarray,
                           lambda a: a.shape == (9, 9),
                           lambda a: a.dtype == 'int64')

In [ ]:
thread_last(raw_example, parse_raw, sudoku_schema.validate)

* minimalist approach with basic data types
* zero boilerplate
* no context on the data structure itself
* harder to explore
* easier to reuse

## Multi-paradigm solution

In [ ]:
@dataclass
class Sudoku:
    grid: np.ndarray
    
    @property
    def remaining_blanks(self):
        return (self.grid == 0).sum()
    
    def __repr__(self):
        ...

* "shallow" class
* saves a lot of boilerplate code
* adds context for user

# State handling - mutable vs. immutable
Example: Fill digits into Sudoku

Using a multi-paradigm implementation, inspired by `pandas`:

In [ ]:
from sudoku.mp.base import Sudoku

blank = 81 * '0'
sudoku = Sudoku.from_string(blank)
sudoku

## Mutable (OO)

In [ ]:
sudoku.set_digit(0, 0, 7, inplace=True)
sudoku

* changed in-place
* seems "natural"
* no way back / history

## Immutable (FP)

In [ ]:
sudoku.set_digit(2, 2, 4, inplace=False)

In [ ]:
sudoku

* easy to reuse or parallelise (efficienct, avoids concurrecny errors)
* natural versioning
* lends itself well to pipelines or method chaining

### Method Chaining

In [ ]:
(sudoku
 .set_digit(2, 8, 9)
 .set_digit(1, 0, 9)
 .set_digit(0, 3, 9))

## Recommendation
* make use of immutable data structures like `@dataclass(frozen=True)`, `NamedTuple`, `frozendict` and `pyrsistent.pmap`)
* use mutable data structures in immutable ways (try the `toolz` library!)
* keep functions pure and idempotent - use classes where configuration and state is required

## Example in Pandas

In [ ]:
import pandas as pd

df = pd.DataFrame(np.random.random((5,3)), columns=list('abc'))
df

In [ ]:
(df
 .assign(sum=lambda df: df.sum(axis=1))
 .assign(a_percent=lambda df: df['a'] / df['sum'])
 .drop(index=[1,3]))

In [ ]:
df

* cleaner Jupyter notebooks (execution order...)
* better reusability
* close to production-ready

# Multiple implementations: polymorphism vs. function composition
Example: Different Sudoku solvers

* Deterministic (mask, fill unambiguous, repeat) - insufficient

* Random (mask, fill random, repeat) - prohibitively slow

* Combined (deterministic as much as possible, random step, repeat)

## OO - Solver class hierarchy

<img src="./img/erm_solver.png" style="max-height: 800px"/>

In [ ]:
from sudoku.oo.solver import *

sudoku = Sudoku.from_string(raw_example)
solver = DeterministicSolver(sudoku)
solver.solve()

sudoku

* Mutable data access (as before)
* Single-method classes excessive (boilerplate!)
* Complicated design for simple functionality

In [ ]:
!wc ./sudoku/oo/solver.py

## FP - solving function composition

<img src="./img/fp_solve.png" style="max-height: 600px;"/>

In [ ]:
from sudoku.fp.solve import *
from sudoku.fp.load import *

solve_combined = partial(solve, step_function=combined_step)

thread_last(raw_example, parse_raw, solve_combined, format_sudoku, print)

* very clear responsibilities per function
* simple, pragmatic design
* easy to introspect
* much more concise (*and* no base module!)

## Multi-paradigm solution

In [ ]:
from sudoku.fp import solve as _fp_solve
from sudoku.mp.base import Sudoku


def solve_sudoku(sudoku: Sudoku, step_function: Callable, max_tries: int = 1):
    if max_tries == 1:
        solved_grid = _fp_solve.solve(sudoku.grid, step_function)
    else:
        solved_grid = _fp_solve.repeat_solve(sudoku.grid,
                                             partial(_fp_solve.solve, step_function=step_function),
                                             max_tries=max_tries)
    return Sudoku(solved_grid)

* simplicity and clarity of FP
* takes and returns high-context Sudoku objects

Or, with more context:

In [ ]:
from sudoku.mp.solve import solve


@dataclass(frozen=True)
class Solver:
    step_function: Callable
    max_tries: int = 1

    def __call__(self, sudoku: Sudoku):
        return solve(sudoku, self.step_function, self.max_tries)

In [ ]:
thread_last(raw_example, 
            Sudoku.from_string, 
            Solver(combined_step, max_tries=100))

# Key Takeaways

## Object-orientation
* "top-down" design
* larger, topical structures
* explicit, high-context
* functionality and data intertwined

leads to:

* intuitive use cases
* high explorability

## Functional programming
* "bottom-up" design
* simplistic thinking
* small chunks of reusable logic, separate from data
* high isolation, low context

leads to
* high reusability
* tidy, concise code
* flexible use cases

## Multi-paradigm programming
*pick & mix* of both worlds:
* pure functions in mutable context
    * brings the simplicity and elegance of FP into OO
    * make your code explorable and easy to understand
    * *remember*: no side effects, no problem!
* mutable data in immutable context
    * use your favourite OO libaries in concise FP code
    * *remember*: copy-and-modify mutable data structures!

leads to (ideally) - best of both worlds:
* intuitive **and** flexible use cases
* high explorability **and** reusability

## My preferred Approach
* iterate with a REPL
* use immutable data types and pure functions where possible
* create classes where either:
    * required due to syntax or library
    * high-context use cases are required

# Thank you for your attention!

# References & Further Reading
Full notebook and code available at https://github.com/eliasmistler/europython2020-multi-paradigm-sudoku

* [MP Patterns](https://www.researchgate.net/publication/2740355_Multiparadigm_Patterns_of_Thought_and_Design)
* [OO Patterns](https://www.oodesign.com/)
* [OO Antipatterns](https://wiki.c2.com/?ClassicOoAntiPatterns)
* [FP Patterns](https://patternsinfp.wordpress.com/)
* [FP Basics](https://www.freecodecamp.org/news/an-introduction-to-the-basic-principles-of-functional-programming-a2c2a15c84/)
* [OO Basics](https://introprogramming.info/english-intro-csharp-book/read-online/chapter-20-object-oriented-programming-principles/)
* [`toolz` library](https://toolz.readthedocs.io/en/latest/)
* [Python dataclasses](https://docs.python.org/3/library/dataclasses.html)
* [`schema` library](https://pypi.org/project/schema/)
* [OO vs FP](https://www.codenewbie.org/blogs/object-oriented-programming-vs-functional-programming)
* [OO imrpoved by non-member functions](https://www.drdobbs.com/cpp/how-non-member-functions-improve-encapsu/184401197)
* [OpenSudoku](https://opensudoku.moire.org/)
* [class or callable?](https://treyhunner.com/2019/04/is-it-a-class-or-a-function-its-a-callable/)
* [Python decorators](https://blog.miguelgrinberg.com/post/the-ultimate-guide-to-python-decorators-part-i-function-registration) and [a primer](https://realpython.com/primer-on-python-decorators/)
* [Python dependency injection](https://medium.com/@shivama205/dependency-injection-python-cb2b5f336dce)